In [1]:
import os
import rasterio
import numpy as np
import pandas as pd
from rasterio.warp import reproject, Resampling

# Define the specific class labels
classes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220]

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2006 to 2015 Exports\Copernicus LC\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Difference"
output_dir = r"C:\Users\llccl\Downloads\1.Difference"

# Create a mapping from class label to index
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process all LST files
for lst_filename in os.listdir(lst_dir):
    if lst_filename.endswith(".tif"):
        lst_path = os.path.join(lst_dir, lst_filename)
        output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_transition_matrix.csv")
        
        # Initialize a 22x22 matrix to store mean values
        transition_matrix = np.full((22, 22), np.nan)
        
        # Open the LST image and extract the data and CRS
        with rasterio.open(lst_path) as lst_ds:
            lst_data = lst_ds.read(1)
            lst_transform = lst_ds.transform
            lst_crs = lst_ds.crs
        
        # Iterate over each file in the mask directory
        for filename in os.listdir(mask_dir):
            if filename.endswith("_Mask.tif"):
                # Parse the class transition from the filename
                parts = filename.split('_')
                class_from = int(parts[0].replace("Class", ""))
                class_to = int(parts[2].replace("Class", ""))
                
                # Check if class_from and class_to are in the specified classes
                if class_from in class_to_index and class_to in class_to_index:
                    # Get corresponding indices for matrix placement
                    from_idx = class_to_index[class_from]
                    to_idx = class_to_index[class_to]
                    
                    # Open the mask image
                    mask_path = os.path.join(mask_dir, filename)
                    with rasterio.open(mask_path) as mask_ds:
                        mask_crs = mask_ds.crs
                        
                        # Reproject if the CRS does not match
                        if mask_crs != lst_crs:
                            reprojected_mask = np.empty_like(lst_data, dtype=np.float32)
                            reproject(
                                source=rasterio.band(mask_ds, 1),
                                destination=reprojected_mask,
                                src_transform=mask_ds.transform,
                                src_crs=mask_crs,
                                dst_transform=lst_transform,
                                dst_crs=lst_crs,
                                resampling=Resampling.nearest
                            )
                        else:
                            reprojected_mask = mask_ds.read(1)
                        
                        # Ensure dimensions match
                        if reprojected_mask.shape != lst_data.shape:
                            print(f"Warning: Mask dimensions {reprojected_mask.shape} do not match LST dimensions {lst_data.shape}. Adjusting mask dimensions...")
                            reprojected_mask = reprojected_mask[:lst_data.shape[0], :lst_data.shape[1]]
                        
                        # Extract LST values where the mask has value 1
                        masked_lst_values = lst_data[reprojected_mask == 1]
                        
                        # Remove NaN values if any
                        masked_lst_values = masked_lst_values[~np.isnan(masked_lst_values)]
                        
                        # Calculate Q1, Q3 and filter values within the IQR range
                        if masked_lst_values.size > 0:
                            q1 = np.percentile(masked_lst_values, 25)
                            q3 = np.percentile(masked_lst_values, 75)
                            filtered_values = masked_lst_values[(masked_lst_values >= q1) & (masked_lst_values <= q3)]
                            
                            # Calculate the mean of the filtered values
                            if filtered_values.size > 0:
                                mean_value = np.mean(filtered_values)
                                transition_matrix[from_idx, to_idx] = mean_value
        
        # Convert matrix to DataFrame and save to CSV
        transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{cls}' for cls in classes], index=[f'Class{cls}' for cls in classes])
        transition_df.to_csv(output_csv, index=True)
        print(f"Processed {lst_filename} and saved transition matrix to {output_csv}")

Processed MOYD21A1DN_2016-2020_2000-2005_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\MOYD21A1DN_2016-2020_2000-2005_Average_transition_matrix.csv
Processed MOYD21A1DN_2016-2020_2000-2005_RegressCombination.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\MOYD21A1DN_2016-2020_2000-2005_RegressCombination_transition_matrix.csv
Processed MOYD21A1D_2016-2020_2000-2005_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\MOYD21A1D_2016-2020_2000-2005_mean_Average_transition_matrix.csv
Processed MOYD21A1N_2016-2020_2000-2005_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\1.Difference\MOYD21A1N_2016-2020_2000-2005_mean_Average_transition_matrix.csv


In [2]:
import os
import rasterio
import numpy as np
import pandas as pd
from rasterio.warp import reproject, Resampling

# Define the specific class labels
classes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220]

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2006 to 2015 Exports\Copernicus LC\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Linear Trend"
output_dir = r"C:\Users\llccl\Downloads\2.LinearTrend"

# Create a mapping from class label to index
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process all LST files
for lst_filename in os.listdir(lst_dir):
    if lst_filename.endswith(".tif"):
        lst_path = os.path.join(lst_dir, lst_filename)
        output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_transition_matrix.csv")
        
        # Initialize a 22x22 matrix to store mean values
        transition_matrix = np.full((22, 22), np.nan)
        
        # Open the LST image and extract the data and CRS
        with rasterio.open(lst_path) as lst_ds:
            lst_data = lst_ds.read(1)
            lst_transform = lst_ds.transform
            lst_crs = lst_ds.crs
        
        # Iterate over each file in the mask directory
        for filename in os.listdir(mask_dir):
            if filename.endswith("_Mask.tif"):
                # Parse the class transition from the filename
                parts = filename.split('_')
                class_from = int(parts[0].replace("Class", ""))
                class_to = int(parts[2].replace("Class", ""))
                
                # Check if class_from and class_to are in the specified classes
                if class_from in class_to_index and class_to in class_to_index:
                    # Get corresponding indices for matrix placement
                    from_idx = class_to_index[class_from]
                    to_idx = class_to_index[class_to]
                    
                    # Open the mask image
                    mask_path = os.path.join(mask_dir, filename)
                    with rasterio.open(mask_path) as mask_ds:
                        mask_crs = mask_ds.crs
                        
                        # Reproject if the CRS does not match
                        if mask_crs != lst_crs:
                            reprojected_mask = np.empty_like(lst_data, dtype=np.float32)
                            reproject(
                                source=rasterio.band(mask_ds, 1),
                                destination=reprojected_mask,
                                src_transform=mask_ds.transform,
                                src_crs=mask_crs,
                                dst_transform=lst_transform,
                                dst_crs=lst_crs,
                                resampling=Resampling.nearest
                            )
                        else:
                            reprojected_mask = mask_ds.read(1)
                        
                        # Ensure dimensions match
                        if reprojected_mask.shape != lst_data.shape:
                            print(f"Warning: Mask dimensions {reprojected_mask.shape} do not match LST dimensions {lst_data.shape}. Adjusting mask dimensions...")
                            reprojected_mask = reprojected_mask[:lst_data.shape[0], :lst_data.shape[1]]
                        
                        # Extract LST values where the mask has value 1
                        masked_lst_values = lst_data[reprojected_mask == 1]
                        
                        # Remove NaN values if any
                        masked_lst_values = masked_lst_values[~np.isnan(masked_lst_values)]
                        
                        # Calculate Q1, Q3 and filter values within the IQR range
                        if masked_lst_values.size > 0:
                            q1 = np.percentile(masked_lst_values, 25)
                            q3 = np.percentile(masked_lst_values, 75)
                            filtered_values = masked_lst_values[(masked_lst_values >= q1) & (masked_lst_values <= q3)]
                            
                            # Calculate the mean of the filtered values
                            if filtered_values.size > 0:
                                mean_value = np.mean(filtered_values)
                                transition_matrix[from_idx, to_idx] = mean_value
        
        # Convert matrix to DataFrame and save to CSV
        transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{cls}' for cls in classes], index=[f'Class{cls}' for cls in classes])
        transition_df.to_csv(output_csv, index=True)
        print(f"Processed {lst_filename} and saved transition matrix to {output_csv}")

Processed MOYD21A1DN_2000-2020_LinearTrend_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\MOYD21A1DN_2000-2020_LinearTrend_Average_transition_matrix.csv
Processed MOYD21A1DN_2000-2020_LinearTrend_RegressCombination.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\MOYD21A1DN_2000-2020_LinearTrend_RegressCombination_transition_matrix.csv
Processed MOYD21A1D_2000-2020_LinearTrend_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\MOYD21A1D_2000-2020_LinearTrend_Average_transition_matrix.csv
Processed MOYD21A1N_2000-2020_LinearTrend_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\2.LinearTrend\MOYD21A1N_2000-2020_LinearTrend_Average_transition_matrix.csv


In [3]:
import os
import rasterio
import numpy as np
import pandas as pd
from rasterio.warp import reproject, Resampling

# Define the specific class labels
classes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220]

# Define directories
mask_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\3. Disturbed Mask\2006 to 2015 Exports\Copernicus LC\Disturbed_Mask_LC"
lst_dir = r"C:\Users\llccl\OneDrive\Documents\To Do\Academics\2nd Year - 1st Semester\Master Thesis\2 - Thesis Proper\5 - Processing\7. LC_LST_Results\4. Compiled_Average_Combination\Mean"
output_dir = r"C:\Users\llccl\Downloads\3.Mean"

# Create a mapping from class label to index
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process all LST files
for lst_filename in os.listdir(lst_dir):
    if lst_filename.endswith(".tif"):
        lst_path = os.path.join(lst_dir, lst_filename)
        output_csv = os.path.join(output_dir, f"{os.path.splitext(lst_filename)[0]}_transition_matrix.csv")
        
        # Initialize a 22x22 matrix to store mean values
        transition_matrix = np.full((22, 22), np.nan)
        
        # Open the LST image and extract the data and CRS
        with rasterio.open(lst_path) as lst_ds:
            lst_data = lst_ds.read(1)
            lst_transform = lst_ds.transform
            lst_crs = lst_ds.crs
        
        # Iterate over each file in the mask directory
        for filename in os.listdir(mask_dir):
            if filename.endswith("_Mask.tif"):
                # Parse the class transition from the filename
                parts = filename.split('_')
                class_from = int(parts[0].replace("Class", ""))
                class_to = int(parts[2].replace("Class", ""))
                
                # Check if class_from and class_to are in the specified classes
                if class_from in class_to_index and class_to in class_to_index:
                    # Get corresponding indices for matrix placement
                    from_idx = class_to_index[class_from]
                    to_idx = class_to_index[class_to]
                    
                    # Open the mask image
                    mask_path = os.path.join(mask_dir, filename)
                    with rasterio.open(mask_path) as mask_ds:
                        mask_crs = mask_ds.crs
                        
                        # Reproject if the CRS does not match
                        if mask_crs != lst_crs:
                            reprojected_mask = np.empty_like(lst_data, dtype=np.float32)
                            reproject(
                                source=rasterio.band(mask_ds, 1),
                                destination=reprojected_mask,
                                src_transform=mask_ds.transform,
                                src_crs=mask_crs,
                                dst_transform=lst_transform,
                                dst_crs=lst_crs,
                                resampling=Resampling.nearest
                            )
                        else:
                            reprojected_mask = mask_ds.read(1)
                        
                        # Ensure dimensions match
                        if reprojected_mask.shape != lst_data.shape:
                            print(f"Warning: Mask dimensions {reprojected_mask.shape} do not match LST dimensions {lst_data.shape}. Adjusting mask dimensions...")
                            reprojected_mask = reprojected_mask[:lst_data.shape[0], :lst_data.shape[1]]
                        
                        # Extract LST values where the mask has value 1
                        masked_lst_values = lst_data[reprojected_mask == 1]
                        
                        # Remove NaN values if any
                        masked_lst_values = masked_lst_values[~np.isnan(masked_lst_values)]
                        
                        # Calculate Q1, Q3 and filter values within the IQR range
                        if masked_lst_values.size > 0:
                            q1 = np.percentile(masked_lst_values, 25)
                            q3 = np.percentile(masked_lst_values, 75)
                            filtered_values = masked_lst_values[(masked_lst_values >= q1) & (masked_lst_values <= q3)]
                            
                            # Calculate the mean of the filtered values
                            if filtered_values.size > 0:
                                mean_value = np.mean(filtered_values)
                                transition_matrix[from_idx, to_idx] = mean_value
        
        # Convert matrix to DataFrame and save to CSV
        transition_df = pd.DataFrame(transition_matrix, columns=[f'Class{cls}' for cls in classes], index=[f'Class{cls}' for cls in classes])
        transition_df.to_csv(output_csv, index=True)
        print(f"Processed {lst_filename} and saved transition matrix to {output_csv}")

Processed MOYD21A1DN_2000-2020_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\MOYD21A1DN_2000-2020_mean_Average_transition_matrix.csv
Processed MOYD21A1DN_2000-2020_mean_RegressCombination.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\MOYD21A1DN_2000-2020_mean_RegressCombination_transition_matrix.csv
Processed MOYD21A1D_2000-2020_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\MOYD21A1D_2000-2020_mean_Average_transition_matrix.csv
Processed MOYD21A1N_2000-2020_mean_Average.tif and saved transition matrix to C:\Users\llccl\Downloads\3.Mean\MOYD21A1N_2000-2020_mean_Average_transition_matrix.csv
